<a href="https://colab.research.google.com/github/SGRGit/gwOsc/blob/FinalCode/DataPrepForBlipGlitchClassification_G.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

In [0]:
import gwpy
import numpy as np
import matplotlib.pyplot as plt
import math

****Analysis with LIGO data for Glitch Recognition using Q-Transforms****

In [0]:
#Import Blip Glitch o1 Data from gspy in Pandas Dataframe
import pandas as pd
raw_data = pd.read_csv("/content/gdrive/My Drive/gspy_blip_o1.csv")

glitch_data = raw_data[['GPStime','ifo', 'peakFreq', 'snr', 'amplitude', 'centralFreq','duration']]

In [0]:
import os
base_dir = "gdrive/My Drive/Colab Notebooks/GlitchNoiseClassification/"
seta = 'Glitch'
setb = 'Noise'

data_dir = os.path.join(base_dir, 'Data')
seta_data_dir = os.path.join(data_dir, seta)
seta_data_fnnames = os.listdir(seta_data_dir)

setb_data_dir = os.path.join(data_dir, setb)
setb_data_fnnames = os.listdir(setb_data_dir)

In [15]:
import os
files = os.listdir(setb_data_dir)
len(files)

500

In [0]:
glitch_data_cln =  glitch_data.iloc[525 : ]

In [0]:
glitch_data_cln.reset_index(inplace = True)

In [0]:
glitch_data_cln

,index,GPStime,ifo,peakFreq,snr,amplitude,centralFreq,duration
0,525,1.132667e+09,H1,170.654,8.714,5.850000e-23,256.501,0.219
1,526,1.130182e+09,H1,262.065,15.209,2.400000e-22,2410.835,0.157
2,527,1.135873e+09,H1,111.128,9.379,1.440000e-22,167.031,0.031
3,528,1.135738e+09,H1,89.676,18.549,5.870000e-22,159.648,0.313
4,529,1.133082e+09,L1,137.711,10.498,9.570000e-23,1564.849,0.375
...,...,...,...,...,...,...,...,...
2590,3115,1.127063e+09,H1,180.507,8.031,1.070000e-22,222.291,0.125
2591,3116,1.128756e+09,H1,312.908,8.195,1.010000e-22,312.274,0.344
2592,3117,1.128127e+09,H1,53.485,8.049,1.140000e-21,83.413,0.094
2593,3118,1.134499e+09,L1,211.477,9.123,8.990000e-23,578.150,0.023


In [0]:
import time
from gwpy.timeseries import TimeSeries
epoch_noise = np.empty(3)
ifo_noise = []
images_noise = []

for i in range(0, 3, 1):
  epoch_noise[i] = int(glitch_data['GPStime'][i])
  ifo_noise.append(glitch_data['ifo'][i])
print(ifo_noise, epoch_noise)

for a in range(0, 3, 1):
  I = TimeSeries.fetch_open_data(ifo_noise[a], epoch_noise[a]-5, epoch_noise[a]-1).q_transform()
  images_noise.append(I)
  fig, ax = plt.subplots(1, 1, figsize = (10, 5))
  ax.imshow(I)
  ax.set_xscale('seconds')
  ax.set_yscale('log')
  ax.set_ylabel('Frequency [Hz]')
  ax.grid(True, axis='y', which='both')
  ax.colorbar(cmap='viridis', label='Normalized energy')
  ax.set_title('Q-Transform for Noise at %s on GPS %d' %(ifo_noise[a], epoch_noise[a]))
  plt.savefig(os.path.join(setb_data_dir,'Q-Transform for Noise at %s on GPS %d' %(ifo_noise[a], epoch_noise[a])))
  time.sleep(0.01)

['L1', 'H1', 'H1'] [1.12719863e+09 1.13221326e+09 1.13620620e+09]


KeyboardInterrupt: ignored

In [0]:
import time
from gwpy.timeseries import TimeSeries
epoch_glitch= np.empty(2)
ifo_glitch = []
interval = np.empty(2)
images_glitch = []

for i in range(0, 2, 1):
  epoch_glitch[i] = int(glitch_data_cln['GPStime'][i])
  ifo_glitch.append(glitch_data_cln['ifo'][i])
  interval[i] = round((glitch_data_cln['GPStime'][i] %1), 3)
print(ifo_glitch, epoch_glitch, interval)

for a in range(0, 2, 1):
  print(a)
  I = TimeSeries.fetch_open_data(ifo_glitch[a], epoch_glitch[a]-2, epoch_glitch[a]+2).q_transform(outseg = (epoch_glitch[a]+interval[a]-0.1 , epoch_glitch[a] +interval[a]+ 0.1))
  images_glitch.append(I)
  fig, ax = plt.subplots(1, 1, figsize = (10, 5))
  ax = plt.gca()
  ax.imshow(I)
  #ax.set_epoch(epoch_glitch[a])
  ax.set_xscale('seconds')
  ax.set_yscale('log')
  ax.set_ylabel('Frequency [Hz]')
  ax.grid(True, axis='y', which='both')
  ax.colorbar(cmap='viridis', label='Normalized energy', clim=[0, 40])
  ax.set_title('Q-Transform for Glitch at %s on GPS %d' %(ifo_glitch[a], epoch_glitch[a]))
  plt.savefig(os.path.join(seta_data_dir,'Q-Transform for Glitch at %s on GPS %d' %(ifo_glitch[a], epoch_glitch[a])))
  plt.close()
  time.sleep(0.01)

['H1', 'H1'] [1.13266652e+09 1.13018160e+09] [0.364 0.02 ]
0
1
